<a href="https://colab.research.google.com/github/niroojen-thambimuthu/SAIT-475/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! zip -s- "/content/drive/My Drive/DATA-475/plant-seedlings-classification.zip" -O plant-seedlings-classification.zip
! unzip plant-seedlings-classification.zip -d plant-seedlings-classification

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

labels = os.listdir("plant-seedlings-classification/train")
print(labels)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


fig, axs = plt.subplots(nrows=3, ncols=4)
fig.tight_layout()
for index, label in enumerate(labels):
    image = np.random.choice(os.listdir(f"plant-seedlings-classification/train/{label}"))
    im = Image.open(f"plant-seedlings-classification/train/{label}/{image}")
    ind = int(index/4), index % 4
    axs[ind].imshow(im)
    axs[ind].set_title(label.replace(" ", "\n"))
    axs[ind].axis("off")

In [ ]:
from fastai.vision import *
from fastai.metrics import accuracy

# Use from_folder factory method from ImageDataBunch to create a dataset
# configure: 
#   path
#   size
#   ds_tfms, to flip the image randomly and allow fliping vertically
#   valid_pct， 20%

path = "/content/plant-seedlings-classification"
# size = 224
# bs = 64

# data = ImageDataBunch.from_folder(
#     path,
#     train="train",
#     test="test",
#     ds_tfms=get_transforms(do_flip=True, flip_vert=True),
#     valid_pct=.2,
#     size=size,
#     bs=bs
# )

data = ImageDataBunch.from_folder(
    "/content/plant-seedlings-classification",
    train="train",
    test="test",
    ds_tfms=get_transforms(do_flip=True, flip_vert=True),
    valid_pct=.2,
    size=224,
    bs=64,
)

In [ ]:
# normalize with imagenet_stats
data.normalize(imagenet_stats)

In [ ]:
# create a CNN learner named "learner" with resnet34
learner = cnn_learner(data, models.resnet34, metrics=[accuracy])

In [ ]:
# find a learning rate
learner.lr_find()
learner.recorder.plot(suggestion=True)

In [ ]:
# fit one cycle
lr = learner.recorder.min_grad_lr
learner.fit_one_cycle(8, lr)

In [ ]:
# interpret the result and see the most confused classess
interp = ClassificationInterpretation.from_learner(learner)
interp.most_confused(min_val=2)

In [ ]:
# fine tune the model and check the result again
interp.plot_confusion_matrix(figsize=(12, 12))